# 作业三：选择模型并完成训练

## 评分标准

* 选择模型，可用套件，也可以自己组网 （30分）

* 配置超参数并训练模型（30分）

* 测试模型效果（30分）

* 可视化模型效果（10分）

## 模型准备

（为了缩短开发周期，建议选择飞桨官方套件，如果有余力可以自己组网实现）

In [ ]:
# model prepare
# 在飞将bml中 https://ai.baidu.com/bml/app/project/script/vis/32035/create
# 选Cascade_Mask_RCNN-RestNet50-FPN 模型


## 模型训练

训练模型需要说明：

* lr_schedule
* optimize
* epoch =50
* batch_size =2
* Loss function 

In [ ]:
# train prepare

## 测试模型

如果可以可视化模型效果则加分

In [ ]:
# test prepare
# api接口：https://aip.baidubce.com/rpc/2.0/ai_custom_bml/v1/segmentation/okkobml
!pip install pycocotools

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.


In [7]:

import json
import base64
import requests
import pycocotools.mask as mask_util
import cv2
import numpy as np
import os
"""
使用 requests 库发送请求
使用 pip（或者 pip3）检查我的 python3 环境是否安装了该库，执行命令
  pip freeze | grep requests
若返回值为空，则安装该库
  pip install requests
"""
# 目标图片的 本地文件路径，支持jpg/png/bmp格式
# 可选的请求参数
# threshold: 默认值为建议阈值，请在 我的模型-模型效果-完整评估结果-详细评估 查看建议阈值
PARAMS = {"threshold": 0.8}
# 服务详情 中的 接口地址
MODEL_API_URL = "https://aip.baidubce.com/rpc/2.0/ai_custom_bml/v1/segmentation/okkobml"
# 调用 API 需要 ACCESS_TOKEN。若已有 ACCESS_TOKEN 则于下方填入该字符串
# 否则，留空 ACCESS_TOKEN，于下方填入 该模型部署的 API_KEY 以及 SECRET_KEY，会自动申请并显示新 ACCESS_TOKEN
ACCESS_TOKEN = ""
API_KEY = "imAtmRRZLyi8AYSGOW8vTqUo"
SECRET_KEY = "ZaLAoE4Oo5Xbsva7IPTvltO6pc8vPsOg"
# 读取视频文件
cap = cv2.VideoCapture('data/footballgood.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
isOpened = cap.isOpened # 判断是否打开‘
# print(isOpened)
# 获取信息 宽高
n_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('总帧数：',n_frame) # 整个视频的总帧数
fps = cap.get(cv2.CAP_PROP_FPS) # 帧率 每秒展示多少张图片
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # w
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # h
print('帧数、宽度、高度分别为：',fps,width,height) # 帧率 宽 高
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('data/result111.mp4', fourcc, fps, (width, height),True)
i = 0 # 记录读取多少帧
frameFrequency = 1 # 每frameFrequency保存一张图片
a = []
number=0
j=0 # 记录读取多少次数
while(isOpened):
    # 结束标志是否读取到最后一帧
    
    if i == n_frame:
        break
    else:
        i = i+1
    (flag,frame) = cap.read() # read方法 读取每一张 flag是否读取成功 frame 读取内容
    fileName = 'image'+str(i)+'.jpg' # 名字累加
    # True表示读取成功 进行·写入
    # if 判断需要有冒号
    #if flag == True:
    outPutDirName = 'data/d/' # 设置保存路径
    if not os.path.exists(outPutDirName):
        # 如果文件目录不存在则创建目录
        os.makedirs(outPutDirName)
    if i % frameFrequency == 0:
        print(fileName)
        cv2.imwrite(outPutDirName+fileName,frame,[cv2.IMWRITE_JPEG_QUALITY,100])# 质量控制 100最高

    IMAGE_FILEPATH = outPutDirName+fileName
    # print("1. 读取目标图片 '{}'".format(IMAGE_FILEPATH))
    with open(IMAGE_FILEPATH, 'rb') as f:
        base64_data = base64.b64encode(f.read())
        base64_str = base64_data.decode('UTF8')
    # print("将 BASE64 编码后图片的字符串填入 PARAMS 的 'image' 字段")
    PARAMS["image"] = base64_str
    
    # print("2. ACCESS_TOKEN 为空，调用鉴权接口获取TOKEN")
    auth_url = "https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials"\
                "&client_id={}&client_secret={}".format(API_KEY, SECRET_KEY)
    auth_resp = requests.get(auth_url)
    auth_resp_json = auth_resp.json()
    ACCESS_TOKEN = auth_resp_json["access_token"]
    # print("新 ACCESS_TOKEN: {}".format(ACCESS_TOKEN))
    # print("3. 向模型接口 'MODEL_API_URL' 发送请求")
    request_url = "{}?access_token={}".format(MODEL_API_URL, ACCESS_TOKEN)
    response = requests.post(url=request_url, json=PARAMS)
    response_json = response.json()
    response_str = json.dumps(response_json, indent=4, ensure_ascii=False)
    # print("结果:\n{}".format(response_str))
    results = json.loads(response_str)['results']
    ori_img = cv2.imread(IMAGE_FILEPATH).astype(np.float32)
    height, width = ori_img.shape[:2]
    
    alpha = 0.8
    for item in results:
        # Draw bbox
        x1 = int(item["location"]["left"])
        y1 = int(item["location"]["top"])
        w = int(item["location"]["width"])
        h = int(item["location"]["height"])
        x2 = x1 + w
        y2 = y1 + h
        # cv2.rectangle(ori_img, (x1, y1), (x2, y2), (0,255,0), 2)
        # cv2.putText(ori_img, "{} score: {}".format(item["name"], round(float(item["score"]),4)), (x1, y1 - 10), cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 255, 255), 1)
        # Draw mask
        rle_obj = {"counts": item['mask'],
                    "size": [height, width]}
        mask = mask_util.decode(rle_obj)
    
        new_rle_obj = mask_util.encode(mask)
    
        random_color = np.array([np.random.random()* 255.0,
                                  np.random.random()* 255.0,
                                  np.random.random()* 255.0])
    
        idx = np.nonzero(mask)
    
        ori_img[idx[0], idx[1], :] *= 1.0 - alpha
        ori_img[idx[0], idx[1], :] += alpha * random_color
    
    # a.append(np.int32(y1))
    # if(j>20):
    #     if a[j]<=a[j-1] and a[j-1]>a[j-2]:
    #             number+=1
    # j+=1
    
    ori_img = ori_img.astype(np.uint8)
    # cv2.putText(ori_img,str(number),(50,50),cv2.FONT_HERSHEY_PLAIN,3.0,(0,0,255),3)
    cv2.imwrite('d:/360Downloads/eee/'+fileName,ori_img,[cv2.IMWRITE_JPEG_QUALITY,100])# 质量控制 100最高
    out.write(ori_img)  # 写入帧
    c = cv2.waitKey(50)
    # print(c)
    if c == 27:  #ESC
        break
print('end!')
out.release()
cv2.waitKey(0)
# cv2.destroyAllWindows()

总帧数： 405
帧数、宽度、高度分别为： 23.808598889138725 640 480
image1.jpg
image2.jpg
image3.jpg
image4.jpg
image5.jpg
image6.jpg
image7.jpg
image8.jpg
image9.jpg
image10.jpg
image11.jpg
image12.jpg
image13.jpg
image14.jpg
image15.jpg


KeyboardInterrupt: 

In [ ]:
response_json

NameError: name 'response_json' is not defined

## 提交方式

本作业提交方式需要同学提供作业项目链接

链接地址：{填写你项目的链接}

注意事项：

* 本作业建议大家新建项目，在新建的项目完成作业（新建项目可以挂载需要的数据集）
* 如何新建项目请参考新手教程，在新建项目完成作业后要记得公开，这样我们才能看到你的作业
* 评分标准以上述为准

## 参考
* [基于PaddleSeg的眼底血管分割](https://aistudio.baidu.com/aistudio/projectdetail/1144243)
* [基于PaddleNLP的小样本学习策略库EFL实现中文科学文献学科分类](https://aistudio.baidu.com/aistudio/projectdetail/2117875)